# 2024 CVE CPE Review 

In [102]:
%%capture
#!rm nvd.jsonl
#!wget -q https://nvd.handsonhacking.org/nvd.jsonl

In [103]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import warnings
import calplot
from itables import init_notebook_mode, show
import itables.options as opt

opt.dom = "tpir"
opt.style = "table-layout:auto;width:auto"
init_notebook_mode(all_interactive=True, connected=True)

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))


logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)

In [104]:
row_accumulator = []

for filename in glob.glob('nvd.jsonl'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data:
            if 'configurations' in entry['cve']:
                for config in entry['cve']['configurations']:
                    for node in config['nodes']:
                        for cpe in node['cpeMatch']:
                            if cpe['vulnerable']:
                                        cve = entry['cve']['id']
                                        try:
                                            published_date = entry['cve']['published']
                                        except KeyError:
                                            published_date = 'Missing_Data'
                                        cpe_string = cpe['criteria']
                                        try:
                                            end = cpe['versionEndExcluding']
                                        except: 
                                            end = 'None'
                                        try:
                                            end2 = cpe['versionEndIncluding']
                                        except: 
                                            end2 = 'None' 
                                        try:
                                            start = cpe['versionStartExcluding']
                                        except: 
                                            start = 'None'
                                        try:
                                            start2 = cpe['versionStartIncluding']
                                        except: 
                                            start2 = 'None'     
                                        new_row = { 
                                        'CVE': cve,
                                        'Published': published_date,
                                        'CPE' : cpe_string,
                                        'StartI' : start,
                                        'StartE' : start2,
                                        'EndI' : end,
                                        'EndE' : end2 
                                        }
                                        row_accumulator.append(new_row)
                                 
                                
      
nvd = pd.DataFrame(row_accumulator)
thisyear = ((nvd['Published'] > '2024-01-01') & (nvd['Published']  < '2025-01-01'))
nvd = nvd.loc[thisyear]
nvd = nvd.sort_values(by=['Published'])

### CVEs With Most CPEs


In [105]:
nvd['CVE'].value_counts().head(20)

CVE
CVE-2024-20433    2434
CVE-2024-20414     461
CVE-2024-0158      388
CVE-2023-43078     368
CVE-2023-43511     359
CVE-2024-23368     343
CVE-2024-33056     330
CVE-2024-33014     325
CVE-2024-42420     320
CVE-2024-47801     320
CVE-2024-43424     320
CVE-2024-47005     320
CVE-2024-45829     320
CVE-2024-45842     320
CVE-2024-47406     320
CVE-2024-48870     320
CVE-2024-47549     320
CVE-2024-21461     313
CVE-2023-33109     313
CVE-2023-43536     312
Name: count, dtype: int64

## Most Common CPEs

In [106]:
nvd['CPE'].value_counts().head(20)

CPE
cpe:2.3:o:linux:linux_kernel:*:*:*:*:*:*:*:*                          8093
cpe:2.3:o:apple:macos:*:*:*:*:*:*:*:*                                  836
cpe:2.3:o:microsoft:windows_server_2022_23h2:*:*:*:*:*:*:*:*           426
cpe:2.3:o:microsoft:windows_server_2022:*:*:*:*:*:*:*:*                412
cpe:2.3:o:microsoft:windows_server_2019:*:*:*:*:*:*:*:*                404
cpe:2.3:o:apple:iphone_os:*:*:*:*:*:*:*:*                              400
cpe:2.3:o:apple:ipados:*:*:*:*:*:*:*:*                                 370
cpe:2.3:o:linux:linux_kernel:6.12:rc1:*:*:*:*:*:*                      355
cpe:2.3:o:microsoft:windows_server_2016:*:*:*:*:*:*:*:*                350
cpe:2.3:o:microsoft:windows_11_22h2:*:*:*:*:*:*:*:*                    323
cpe:2.3:a:gitlab:gitlab:*:*:*:*:enterprise:*:*:*                       316
cpe:2.3:o:linux:linux_kernel:6.12:rc2:*:*:*:*:*:*                      313
cpe:2.3:o:microsoft:windows_11_21h2:*:*:*:*:*:*:*:*                    311
cpe:2.3:a:adobe:experience_manager:*:*:*:*:aem_cloud_service:*:*:*     311
cpe:2.3:o:microsoft:windows_10_21h2:*:*:*:*:*:*:*:*                    305
cpe:2.3:o:microsoft:windows_11_23h2:*:*:*:*:*:*:*:*                    302
cpe:2.3:o:microsoft:windows_10_22h2:*:*:*:*:*:*:*:*                    289
cpe:2.3:o:microsoft:windows_server_2012:r2:*:*:*:*:*:*:*               286
cpe:2.3:a:gitlab:gitlab:*:*:*:*:community:*:*:*                        266
cpe:2.3:o:linux:linux_kernel:6.12:rc3:*:*:*:*:*:*                      263
Name: count, dtype: int64

In [107]:
nvd['CPE'].value_counts().describe()

count    19807.000000
mean         5.714545
std         60.042120
min          1.000000
25%          1.000000
50%          1.000000
75%          4.000000
max       8093.000000
Name: count, dtype: float64